In [1]:
import numpy as np
import time
import math

In [2]:
def Pr(Pattern, motif):
    Symbol = ['A','C','G','T']
    probability = 1.0
    for i in range(len(Pattern)):
        probability *= float(motif[:,i][Symbol.index(Pattern[i])])
    return probability

In [3]:
def MaximumFinding(skew):
    i = 0
    index = []
    while max(skew) in skew[i:]:
        j = skew[i:].index(max(skew))
        i += j+1
        index.append(i-1)
    return index

In [4]:
def Entropy(column):
    result  = 0
    for i in column:
        if i == 0:
            result += 0
        else:
            result += -i*math.log2(i)
    return result

def Score(motifs):
    result = 0
    eachEntropy = []
    for i in range(len(motifs[0])):
        result += Entropy(motifs[:,i])
        eachEntropy.append(Entropy(motifs[:,i]))
    return result

In [5]:
def ProfileMotif(Motif):
    row = len(Motif)
    column = len(Motif[0])
    matrix = []
    for i in range(row):
        matrix += list(Motif[i])
    matrix = np.array(matrix).reshape(row,column)
    profile = np.zeros((4,column))
    for i in range(column):  #pseudocount = 1
        profile[:,i][0] = (np.sum(matrix[:,i]=='A')+1)/(row+4)
        profile[:,i][1] = (np.sum(matrix[:,i]=='C')+1)/(row+4)
        profile[:,i][2] = (np.sum(matrix[:,i]=='G')+1)/(row+4)
        profile[:,i][3] = (np.sum(matrix[:,i]=='T')+1)/(row+4)
    return profile

In [17]:
Motif = ['GCC']
ProfileMotif(Motif)

array([[ 0.2,  0.2,  0.2],
       [ 0.2,  0.4,  0.4],
       [ 0.4,  0.2,  0.2],
       [ 0.2,  0.2,  0.2]])

In [6]:
def ProfileMostProbablekmer(Genome, k, ProfileMotif):
    probable = []
    Pattern = []
    for i in range(len(Genome)-k+1):
        probable.append(Pr(Genome[i:i+k],ProfileMotif))
    index = MaximumFinding(probable)
    for j in index:
        Pattern.append(Genome[j:j+k])
    return Pattern[0]

In [12]:
def newGreedyMotifSearch(Dna, k, t):
    BestMotifs = []
    for i in range(t):
        BestMotifs.append(Dna[i][0:k])
    for i in range(len(Dna[0])-k+1):
        Motifs = []
        Motifs.append(Dna[0][i:i+k])
        for j in range(1,t):
            ProfileMotifs = ProfileMotif(Motifs)
            kmer = ProfileMostProbablekmer(Dna[j], k, ProfileMotifs)
            Motifs.append(kmer)
        if Score(ProfileMotif(Motifs)) < Score(ProfileMotif(BestMotifs)):
            BestMotifs = Motifs
            #print(BestMotifs)
    return BestMotifs

In [18]:
info = []
with open('/Users/igao/Documents/bioinformatics/data/dataset_160_9.txt') as f:
    for line in f.readlines():
        info.append(line.strip())
k = int(info[0].split()[0])
t = int(info[0].split()[1])
Dna = info[1:]

In [13]:
k = 3
t = 5
Dna = ['GGCGTTCAGGCA','AAGAATCAGTCA','CAAGGAGTTCGC','CACGTCAATCAC','CAATAATATTCG']

In [19]:
start = time.clock()
result = newGreedyMotifSearch(Dna, k, t)
end = time.clock()
print("run: %fs" % (end-start))

run: 9.045946s


In [16]:
newGreedyMotifSearch(Dna, k, t)

['CAG', 'CAG', 'CAA', 'CAA', 'CAA']

In [23]:
output = ''
for i in result:
    output += i+'\'

SyntaxError: EOL while scanning string literal (<ipython-input-23-86cf16c9a82e>, line 3)

In [24]:
print(output)

GCCTAGTACTCC
GCCTCCTCGTCC
GCCTATTTTTCC
GCCTCCTGCTCC
GCCTTGTTCTCC
GCCTCTTTGTCC
GCCTTCTTTTCC
GCCTCGTTTGCC
GCCTGTTTGTCC
GCCTCGTGATCC
GCCTCCTGTTCC
GCCTACTCTTCC
GCCTTTTACTCC
GCCTATTATTCC
GCCTCTTCATCC
GCCTGATCCTCC
GCCTACTGATCC
GCCTCCTGATCC
GCCTGATAATCC
GCCTAGTGATCC
GCCTTCTCATCC
GCCTATTGTTCC
GCCTGGTTTTCC
GCCTTGTTGTCC
GCCTGTTCATCC

